In [23]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('database.sqlite')

table_names = [
    'Player', 'Extra_Runs', 'Batsman_Scored', 'Batting_Style',
    'Bowling_Style', 'Country', 'Season', 'City', 'Outcome',
    'Win_By', 'Wicket_Taken', 'Venue', 'Extra_Type', 'Out_Type',
    'Toss_Decision', 'Umpire', 'Team', 'Ball_by_Ball', 'Match',
    'Rolee', 'Player_Match'
]

all_tables = {}

for table in table_names:
    try:
        all_tables[table] = pd.read_sql_query(f"SELECT * FROM {table}", connection)
        print(f"The table '{table}' uploaded successfully. Rows: {len(all_tables[table])}")
    except Exception as e:
        print(f"Error '{table}': {e}")



The table 'Player' uploaded successfully. Rows: 469
The table 'Extra_Runs' uploaded successfully. Rows: 7469
The table 'Batsman_Scored' uploaded successfully. Rows: 133097
The table 'Batting_Style' uploaded successfully. Rows: 2
The table 'Bowling_Style' uploaded successfully. Rows: 14
The table 'Country' uploaded successfully. Rows: 12
The table 'Season' uploaded successfully. Rows: 9
The table 'City' uploaded successfully. Rows: 29
The table 'Outcome' uploaded successfully. Rows: 3
The table 'Win_By' uploaded successfully. Rows: 4
The table 'Wicket_Taken' uploaded successfully. Rows: 6727
The table 'Venue' uploaded successfully. Rows: 35
The table 'Extra_Type' uploaded successfully. Rows: 5
The table 'Out_Type' uploaded successfully. Rows: 9
The table 'Toss_Decision' uploaded successfully. Rows: 2
The table 'Umpire' uploaded successfully. Rows: 52
The table 'Team' uploaded successfully. Rows: 13
The table 'Ball_by_Ball' uploaded successfully. Rows: 136590
The table 'Match' uploaded s

**Query 1: Select All Columns from Player’s Table**

Write and execute a SQL query to select all columns from the Player_Match table.

In [36]:
q1 = "SELECT * FROM Player_Match"
df_q1 = pd.read_sql_query(q1, connection)
df_q1

,Match_Id,Player_Id,Role_Id,Team_Id
0,335987,1,1,1
1,335987,2,3,1
2,335987,3,3,1
3,335987,4,3,1
4,335987,5,3,1
...,...,...,...,...
12689,981024,385,3,11
12690,981024,394,3,11
12691,981024,429,3,11
12692,981024,434,3,2


**Query 2: Batsman vs Runs**

Write and execute a SQL query to calculate the total runs scored by each batsman.

In [35]:
q2 = '''
SELECT
    p.Player_Name,
    SUM(bs.Runs_Scored) AS Total_Runs
FROM Ball_by_Ball b
INNER JOIN Player p ON b.Striker = p.Player_Id
INNER JOIN Batsman_Scored bs ON
    b.Match_Id = bs.Match_Id AND
    b.Over_Id = bs.Over_Id AND
    b.Ball_Id = bs.Ball_Id AND
    b.Innings_No = bs.Innings_No
GROUP BY p.Player_Name
ORDER BY Total_Runs DESC;
'''
df_q2 = pd.read_sql_query(q2, connection)
print(df_q2.head(10))

      Player_Name  Total_Runs
0        SK Raina        4106
1         V Kohli        4105
2       RG Sharma        3874
3       G Gambhir        3634
4        CH Gayle        3447
5      RV Uthappa        3390
6       DA Warner        3373
7        MS Dhoni        3270
8  AB de Villiers        3270
9        S Dhawan        3082


**Query 3: Fifties and Hundreds**

Write and execute a SQL query to calculate the number of fifties and hundreds scored by each batsman.

In [33]:
query3 = '''
SELECT
    Player_Name,
    SUM(case when Match_Runs >= 50 Then 1 else 0 end) AS Fifty,
	sum(case when MAtch_runs >=100 then 1 else 0 end) as hundreds
FROM (SELECT
		p.Player_Name,
		bs.match_id,
		sum(bs.Runs_Scored) as MAtch_Runs
FROM Batsman_Scored bs
INNER JOIN Ball_by_Ball b
ON
    b.Match_Id = bs.Match_Id AND
    b.Over_Id = bs.Over_Id AND
    b.Ball_Id = bs.Ball_Id AND
    b.Innings_No = bs.Innings_No
INNER JOIN Player p on p.Player_Id = b.Striker
GROUP BY p.Player_Name, bs.Match_Id
) as MAtchTotal
group by Player_Name
ORDER BY Hundreds DESC, Fifty desc;
'''
df_q3 = pd.read_sql_query(query3, connection)
print(df_q3.head(10))

      Player_Name  Fifty  hundreds
0        CH Gayle     26         5
1         V Kohli     31         4
2  AB de Villiers     25         3
3       DA Warner     34         2
4        V Sehwag     18         2
5       SR Watson     16         2
6         M Vijay     15         2
7    AC Gilchrist     13         2
8     BB McCullum     13         2
9       RG Sharma     30         1


**Query 4: Best Bowling Figures**

Write and execute a SQL query to find the best bowling figures for each bowler.

In [34]:
query4 = '''
SELECT
    Player_Name,
    MAX(Wickets_Count || '-' || Runs_Conceded) AS Best_Figure
FROM (
    SELECT
        p.Player_Name,
        bbb.Match_Id,
        COUNT(wt.Kind_Out) AS Wickets_Count,
        SUM(COALESCE(bs.Runs_Scored, 0)) AS Runs_Conceded
    FROM Ball_by_Ball bbb
    JOIN Player p ON bbb.Bowler = p.Player_Id
    LEFT JOIN Wicket_Taken wt ON bbb.Match_Id = wt.Match_Id
        AND bbb.Over_Id = wt.Over_Id AND bbb.Ball_Id = wt.Ball_Id
        AND wt.Kind_Out != 'run out' -- Ран-ауты не считаются боулеру
    LEFT JOIN Batsman_Scored bs ON bbb.Match_Id = bs.Match_Id
        AND bbb.Over_Id = bs.Over_Id AND bbb.Ball_Id = bs.Ball_Id
    GROUP BY p.Player_Name, bbb.Match_Id
) AS MatchStats
GROUP BY Player_Name
ORDER BY Best_Figure DESC;
'''
df_q4 = pd.read_sql_query(query4, connection)
print(df_q4.head(10))

       Player_Name Best_Figure
0       KA Pollard        9-87
1         B Geeves        9-72
2         DJ Bravo        9-71
3        VY Mahesh        9-70
4        S Kaushik        9-67
5  NM Coulter-Nile        9-66
6        MM Sharma        9-66
7         RP Singh        9-65
8        YS Chahal        9-63
9         Umar Gul        9-63


**Query 5: Comprehensive Career Metrics**

Combine all the previous chunks into a single comprehensive query to get detailed career metrics for players.

In [37]:
query5 = '''
WITH BattingStats AS (
    SELECT
        Player_Name,
        Player_Id,
        SUM(Match_Runs) AS Career_Runs,
        SUM(CASE WHEN Match_Runs >= 50 AND Match_Runs < 100 THEN 1 ELSE 0 END) AS Fifties,
        SUM(CASE WHEN Match_Runs >= 100 THEN 1 ELSE 0 END) AS Hundreds
    FROM (
        SELECT p.Player_Name, p.Player_Id, bbb.Match_Id, SUM(bs.Runs_Scored) AS Match_Runs
        FROM Ball_by_Ball bbb
        JOIN Player p ON bbb.Striker = p.Player_Id
        JOIN Batsman_Scored bs ON bbb.Match_Id = bs.Match_Id
            AND bbb.Over_Id = bs.Over_Id AND bbb.Ball_Id = bs.Ball_Id
            AND bbb.Innings_No = bs.Innings_No
        GROUP BY p.Player_Id, bbb.Match_Id
    )
    GROUP BY Player_Id
),
BowlingStats AS (
    SELECT
        p.Player_Id,
        MAX(Wickets_In_Match || ' / ' || Runs_In_Match) AS Best_Bowling
    FROM (
        SELECT
            bbb.Bowler, bbb.Match_Id,
            COUNT(wt.Kind_Out) AS Wickets_In_Match,
            SUM(COALESCE(bs.Runs_Scored, 0)) AS Runs_In_Match
        FROM Ball_by_Ball bbb
        LEFT JOIN Wicket_Taken wt ON bbb.Match_Id = wt.Match_Id
            AND bbb.Over_Id = wt.Over_Id AND bbb.Ball_Id = wt.Ball_Id
            AND wt.Kind_Out != 'run out'
        LEFT JOIN Batsman_Scored bs ON bbb.Match_Id = bs.Match_Id
            AND bbb.Over_Id = bs.Over_Id AND bbb.Ball_Id = bs.Ball_Id
        GROUP BY bbb.Bowler, bbb.Match_Id
    ) AS MatchStats
    JOIN Player p ON MatchStats.Bowler = p.Player_Id
    GROUP BY p.Player_Id
)
SELECT
    p.Player_Name,
    COALESCE(bat.Career_Runs, 0) AS Total_Runs,
    COALESCE(bat.Fifties, 0) AS Fifties,
    COALESCE(bat.Hundreds, 0) AS Hundreds,
    COALESCE(bowl.Best_Bowling, 'N/A') AS Best_Bowling_Figures
FROM Player p
LEFT JOIN BattingStats bat ON p.Player_Id = bat.Player_Id
LEFT JOIN BowlingStats bowl ON p.Player_Id = bowl.Player_Id
WHERE bat.Career_Runs > 0 OR bowl.Best_Bowling IS NOT NULL
ORDER BY Total_Runs DESC;
'''
df_q5 = pd.read_sql_query(query5, connection)
print(df_q5.head(10))

      Player_Name  Total_Runs  Fifties  Hundreds Best_Bowling_Figures
0        SK Raina        4106       28         1               8 / 63
1         V Kohli        4105       27         4               8 / 56
2       RG Sharma        3874       29         1               8 / 45
3       G Gambhir        3634       31         0                  N/A
4        CH Gayle        3447       21         5               8 / 70
5      RV Uthappa        3390       17         0                  N/A
6       DA Warner        3373       32         2                  N/A
7        MS Dhoni        3270       16         0                  N/A
8  AB de Villiers        3270       22         3                  N/A
9        S Dhawan        3082       25         0                4 / 9
